In [14]:
# after we install all the packages, import all of them for the use in today's lecture!
import wrds as wrds
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [15]:
db = wrds.Connection(wrds_username='marzipan')


Loading library list...
Done


In [16]:
db.create_pgpass_file()

In [17]:
params = {}
params['secids'] = ('108105',)

sql = '''
select date, AVG(impl_volatility) as iv  
from optionm.vsurfd%(year)s
where secid in %(secids)s and days = 30 and abs(delta)<=50
group by date
'''

params['year'] = 2000
data           = db.raw_sql(sql,params = params)

for y in range(2001,2021):
    print('Processing year ', y)
    params['year'] = y
    data = pd.concat([data, db.raw_sql(sql,params = params)])
    
data.loc[:,'date'] = pd.to_datetime(data.loc[:,'date'])

data = data.sort_values('date')

Processing year  2001
Processing year  2002
Processing year  2003
Processing year  2004
Processing year  2005
Processing year  2006
Processing year  2007
Processing year  2008
Processing year  2009
Processing year  2010
Processing year  2011
Processing year  2012
Processing year  2013
Processing year  2014
Processing year  2015
Processing year  2016
Processing year  2017
Processing year  2018
Processing year  2019
Processing year  2020


In [18]:
data

,date,iv
82,2000-01-03,0.223436
165,2000-01-04,0.247982
55,2000-01-05,0.241334
47,2000-01-06,0.238140
65,2000-01-07,0.201195
...,...,...
4,2020-12-24,0.180977
84,2020-12-28,0.183228
187,2020-12-29,0.195475
191,2020-12-30,0.190608


In [9]:
params = {}
params['secids'] = ('108105',)
sql = '''
select a.date, a.return 
from optionm.secprd%(year)s as a
where secid in %(secids)s
'''

params['year'] = 2000
ret           = db.raw_sql(sql,params = params)

for y in range(2001,2021):
    print('Processing year ', y)
    params['year'] = y
    ret = pd.concat([ret, db.raw_sql(sql,params = params)])
    
ret.loc[:,'date'] = pd.to_datetime(ret.loc[:,'date'])

data = pd.merge(data, ret, on =['date'])

Processing year  2001
Processing year  2002
Processing year  2003
Processing year  2004
Processing year  2005
Processing year  2006
Processing year  2007
Processing year  2008
Processing year  2009
Processing year  2010
Processing year  2011
Processing year  2012
Processing year  2013
Processing year  2014
Processing year  2015
Processing year  2016
Processing year  2017
Processing year  2018
Processing year  2019
Processing year  2020


In [10]:
data

,date,iv,return
0,2000-01-03,0.223436,-0.009549
1,2000-01-04,0.247982,-0.038345
2,2000-01-05,0.241334,0.001922
3,2000-01-06,0.238140,0.000956
4,2000-01-07,0.201195,0.027090
...,...,...,...
5279,2020-12-24,0.180977,0.003537
5280,2020-12-28,0.183228,0.008723
5281,2020-12-29,0.195475,-0.002227
5282,2020-12-30,0.190608,0.001342


In [11]:
data.loc[:,'rvar'] = data['return'].rolling(window=30).var()*252
data.loc[:,'vrp'] = data.loc[:,'iv']**2 - data.loc[:,'rvar']
data.loc[:,'fret30d'] = data.loc[:,'return']\
                        .rolling(window=30)\
                        .apply(lambda x: np.nanprod(1+x)-1).shift(-30)
data.loc[:,'constant'] = 1.0
data = data.dropna(how = 'any')
data = data.set_index('date')
data = data.groupby(pd.Grouper(freq='BM')).last()

In [12]:
data

,iv,return,rvar,vrp,fret30d,constant
date,,,,,,
2000-02-29,0.219573,0.013627,0.045022,0.003190,0.098192,1.0
2000-03-31,0.228117,0.007164,0.069520,-0.017483,-0.030842,1.0
2000-04-28,0.237739,-0.008526,0.081109,-0.024589,-0.004430,1.0
2000-05-31,0.225846,-0.001301,0.064648,-0.013642,0.052965,1.0
2000-06-30,0.186941,0.008465,0.041053,-0.006106,0.025410,1.0
...,...,...,...,...,...,...
2020-07-31,0.204348,0.007671,0.029928,0.011830,0.034370,1.0
2020-08-31,0.213516,-0.002195,0.009640,0.035949,0.003321,1.0
2020-09-30,0.243819,0.008254,0.045347,0.014101,0.062340,1.0


In [13]:
Y = data['fret30d']
X = data[['constant','vrp']]
res = sm.OLS(Y,X).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                fret30d   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     2.102
Date:                Tue, 25 May 2021   Prob (F-statistic):              0.148
Time:                        15:37:53   Log-Likelihood:                 362.45
No. Observations:                 250   AIC:                            -720.9
Df Residuals:                     248   BIC:                            -713.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
constant       0.0071      0.004      1.978      0.049    2.98e-05       0.014
vrp            0.1141      0.079      1.450      0.148      -0.041       0.269
==============================================================================
Omnibus:                       59.248   Durbin-Watson:                   1.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              153.443
Skew:                          -1.066   Prob(JB):                     4.79e-34
Kurtosis:                       6.192   Cond. No.                         21.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""